In [18]:
import numpy as np
import ipywidgets as widgets
from ipyleaflet import Map, ImageOverlay
import rasterio as rio
import os
from PIL import Image
import glob
center = (53.8008, -1.5491)
img_counter = 0

m = Map(center=center, zoom=10)


def model_airtemp(solar_irradiance, ndvi, ndbi, ndwi, c=25, ndvi_beta=-3, ndbi_beta=4, ndwi_beta=-2):
    airtemp =  ndvi_beta*ndvi + ndbi_beta*ndbi + ndwi_beta*ndwi + np.random.normal(-1, 1) + c + solar_irradiance
    return airtemp

ndvi_array = rio.open('Data/leeds_NDVI_aug_highres.tif').read()
ndwi_array = rio.open('Data/leeds_NDWI_aug_highres.tif').read()
ndbi_array = rio.open('Data/leeds_NDBI_aug_highres.tif').read()

image_extent = rio.open('Data/leeds_NDVI_aug_highres.tif').bounds
SW = (image_extent.bottom, image_extent.left)
NE = (image_extent.top, image_extent.right)
bounds_tuple = (SW, NE)


def array_to_png(array, filename='temp.png'):
    array = np.moveaxis(array, 0, -1)
    nan_mask = ~np.isnan(array) * 1 
    nan_mask *= 255
    nan_mask = nan_mask.astype(np.uint8)
    array_max = np.nanmax(array)
    array_min = np.nanmin(array)


    array = np.nan_to_num(array)


    array = np.clip((array - array_min) / (array_max - array_min) * 255, 0, 255)
    array = array.astype(np.uint8)
    
    try:
        os.remove(filename)
    except:
        pass
    image = Image.fromarray(np.squeeze(np.stack([array, array, array, nan_mask], axis=-1)), mode="RGBA")
    image.save(filename)
    return filename


solar_irradiance_slider = widgets.FloatSlider(value=14, min=10, max=20, step=0.1, description='Solar Irradiance')
ndvi_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='NDVI Coefficient')
ndbi_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='NDBI Coefficient')
ndwi_slider = widgets.FloatSlider(value=0, min=-10, max=10, step=0.01, description='NDWI Coefficient')

def updateMap(change):

    new_airtemp = model_airtemp(solar_irradiance_slider.value, ndvi_array, ndbi_array, ndwi_array, 13, ndvi_slider.value, 
                  ndbi_slider.value, ndwi_slider.value)
    
    try:
        for layer in m.layers[1:]:
            m.remove_layer(layer)
    except:
        pass    

    random_id = np.random.randint(0, 100000)
    random_path = f'{random_id}.png'
    path = array_to_png(new_airtemp, random_path)

    old_imgs = glob.glob("*.png")

    for img in old_imgs:
        if img != random_path:
            try:
                os.remove(img)
            except:
                pass



    imageLayer = ImageOverlay(url=path, bounds=bounds_tuple)
    m.add_layer(imageLayer)
    

solar_irradiance_slider.observe(updateMap, 'value')
ndvi_slider.observe(updateMap, 'value')
ndbi_slider.observe(updateMap, 'value')
ndwi_slider.observe(updateMap, 'value')


irradiance_slider_container = widgets.Box([solar_irradiance_slider])
parameters_container = widgets.VBox([ndvi_slider, ndbi_slider, ndwi_slider])

controls_container = widgets.VBox([irradiance_slider_container, parameters_container])
mapDisplay = widgets.Output()

mapLayout = widgets.HBox([m, controls_container])



display(mapLayout)


29.126102
26.515278
30.353657
25.397438
27.82486
26.332165
27.95463
26.446945
26.359722
24.76783
27.147963
25.226604
26.962166
24.556028
28.536606
25.195625
29.294333
24.468182
30.121178
24.640625
29.65472
23.979067
29.264317
23.71873
29.058968
24.236881
27.833912
23.601194
29.233927
25.326378
29.87375
26.1613
26.379757
24.378017
27.847553
25.886005
